In [2]:
import scipy.io
import numpy as np


In [3]:
hw6data = scipy.io.loadmat('hw6data.mat')

# Q(b)

In [4]:
hw6data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'L', 'seq1_h', 'seq1_x', 'seq2_h', 'seq2_x'])

In [5]:
x = hw6data['seq1_x']
h = hw6data['seq1_h']
x.shape,h.shape

((8820, 8), (8820, 1))

In [6]:
#estimate A
n11,n12,n21,n22=0,0,0,0
t1,t2=0,0
for i in range(h.shape[0]-1):
    if h[i]==1 and h[i+1]==1:
        n11+=1
        t1+=1
    elif h[i]==1 and h[i+1]==2:
        n12+=1
        t1+=1
    elif h[i]==2 and h[i+1]==1:
        n21+=1
        t2+=1
    elif h[i]==2 and h[i+1]==2:
        n22+=1
        t2+=1

row1 = [n11*1./t1,n12*1./t1]
row2 = [n21*1./t2,n22*1./t2]
A = np.array([row1,row2])
A

array([[ 0.89530603,  0.10469397],
       [ 0.10149788,  0.89850212]])

In [154]:
#estimate mu
mu1,mu2=0,0
n1,n2=0,0
for i in range(x.shape[0]-1):
    if h[i]==1:
        mu1+=x[i]
        n1+=1
    else:
        mu2+=x[i]
        n2+=1
mu1 = mu1/n1
mu2 = mu2/n2


In [155]:
mu1

array([ 3.62234937,  0.4784942 ,  0.71621638,  0.00745382, -0.41282949,
       -1.01921028,  0.08404994, -0.25025128])

In [156]:
mu2

array([ 3.7172099 , -0.49258531, -0.75389926, -0.22607191,  0.25266434,
        0.91925733, -0.0773829 ,  0.24698484])

# Q(C)

Since this is an online decision making, we don't need to consider message passing of 
$m_{t+1}\;(h_t)$. The conditional probability of $p(h_{t}\;|\;\bar{x}_1,..,\bar{x}_t)$ can be written as (for t>1)


$$m_{h(t-1)} \;\;\; (h_t) \times m_{x(t)} \;\;(h_t)$$

For t=1,$p(h_1|x_1) = p(h_1) \times p(\bar{x}_1\;|h_1) $

In [140]:
x = hw6data['seq2_x']
h = hw6data['seq2_h']
x.shape,h.shape

((2205, 8), (2205, 1))

In [141]:
p1 = np.array([A[1][0],A[1][1]])
print("The probability of h1 = 1 is {}, h1=2 is {}".format(p1[0],p1[1]))

The probability of h1 = 1 is 0.10149787614576347, h1=2 is 0.8985021238542366


In [142]:
import numpy as np
from numpy.linalg import norm

In [143]:
np.pi, np.exp(1)

(3.141592653589793, 2.7182818284590451)

In [167]:
# define pass from evidence
def evidence_pass(x,h):
    """
    calculate the pass from x_t to h_t
    h_t = 1 or 2
    """
    mu = mu1 if h == 1 else mu2
    d = len(x)*1.
    return (1./(2*np.pi))**(d/2)*np.exp(-0.5*np.dot(x-mu,x-mu))
    

In [168]:
# define phi for hidden states
phi = []
phi.append([A[1][0],A[1][1]])
for i in range(1,x.shape[0]):
    phi.append([1,1])

In [169]:
phi[0][1]

0.89850212385423656

In [170]:
# pred is the final prediction
pred = []

# base case t=1
# p(h1|x1) = p(h1)p(x1|h1=1)
p11 = p1[0]*evidence_pass(x[0],1)
p12 = p1[1]*evidence_pass(x[0],2)
print(p11,p12)
if p11 > p12:
    pred.append(1)
else:
    pred.append(2)

#base case t = 2
# p(h2|x1,x2) = m(h1->h2)*m(x2->h2)=\sum_h1 p(h1)p(h2|h1)*m(x1->h1)*m(x2->h2)
p21,p22=0,0
for h1 in [1,2]:
    p21+=p1[h1-1]*A[h1-1][0]*evidence_pass(x[0],h1)*evidence_pass(x[1],1)
    p22+=p1[h1-1]*A[h1-1][1]*evidence_pass(x[0],h1)*evidence_pass(x[1],2)
print(p21,p22)

#check message from h1->h2
#m12 = \sum_h1 p(h1)p(h2|h1)p(x1|h1)
m12 = [0,0]
for h1 in [1,2]:
    m12[0] += p1[h1-1]*A[h1-1][0]*evidence_pass(x[0],h1)
    m12[1] += p1[h1-1]*A[h1-1][1]*evidence_pass(x[0],h1)
print(m12)

4.00443578631e-13 2.07979248427e-08
1.24883081886e-16 3.42629398825e-15
[2.1113037193200749e-09, 1.8687021566926809e-08]


In [171]:
def state_pass(i):
    """
    calclulate the forward message pass from state i to i+1
    the result will be stored in the ndarray state_pass_prev
    the formular is m(i,i+1)=\sum_hi phi(hi)*p(hi+1|hi)*m(i-1,i)*m(x(i-1)->h(i-1))
    """
    # calculate m(h1->h2)
    m = [0,0]
    if i == 1:
        for h in [1,2]:
            m[0]+=phi[i-1][h-1]*A[h-1][0]*evidence_pass(x[0],h)
            m[1]+=phi[i-1][h-1]*A[h-1][1]*evidence_pass(x[0],h)
    else:
        for h in [1,2]:
            m[0]+=phi[i-1][h-1]*A[h-1][0]*evidence_pass(x[i-1],h)*state_pass_prev[i-2][0]
            m[1]+=phi[i-1][h-1]*A[h-1][1]*evidence_pass(x[i-1],h)*state_pass_prev[i-2][1]
    return m

In [172]:
# initialize all the message pass between hidden states as 0
state_pass_prev = [[0,0]]*(x.shape[0]-1)
for i in range(1,x.shape[0]):
    state_pass_prev[i-1]=state_pass(i)

In [173]:
state_pass_prev

[[2.1113037193200749e-09, 1.8687021566926809e-08],
 [1.5109949669881344e-16, 3.1942540858292295e-15],
 [6.6590774394604799e-23, 5.2657749735171103e-21],
 [5.4551751844319484e-31, 5.0531707805283683e-30],
 [7.2566582954249891e-42, 7.8907689695841425e-42],
 [4.9725642942922558e-53, 6.3260275755385519e-54],
 [6.2619099694317707e-66, 7.015253328673719e-66],
 [3.9482174284998924e-73, 2.8435715771185691e-72],
 [2.3976589521191987e-79, 1.396029803559349e-77],
 [1.4175842146441755e-87, 7.3052149102500055e-85],
 [5.9806370165590886e-101, 2.6305835956917975e-97],
 [2.7522985048776596e-111, 1.0012441035314694e-106],
 [1.4932190893910346e-118, 2.9640831504143219e-113],
 [8.5045424306481014e-127, 1.4173991581571102e-120],
 [1.4166212114599054e-133, 2.0837545406602485e-126],
 [5.759352740091857e-140, 5.679384800732399e-132],
 [2.3221764667157778e-150, 2.9397104432981953e-143],
 [6.3153573414305474e-158, 7.0302370168213935e-150],
 [5.5828909466993185e-165, 5.4841463311087381e-156],
 [1.21706833688627

In [174]:
# calculate the conditional probability and make prediction
def prediction(i):
    p1,p2=0,0
    if i == 1:
        p1 = phi[i-1][0]*evidence_pass(x[i-1],1)
        p2 = phi[i-1][1]*evidence_pass(x[i-1],2)
    else:
        p1 = phi[i-1][0]*evidence_pass(x[i-1],1)*state_pass_prev[i-2][0]
        p2 = phi[i-1][1]*evidence_pass(x[i-1],2)*state_pass_prev[i-2][1]
    return 1 if p1>p2 else 2

In [175]:
pred = []
for i in range(1,x.shape[0]+1):
    pred.append(prediction(i))

In [176]:
mistake = 0
for i in range(x.shape[0]):
    if pred[i] != h[i]:
        mistake += 1
mistake

1127